In [3]:
import numpy as np
import pandas as pd
import opensmile


import pydub
import librosa
from scipy import signal
from mutagen.mp3 import MP3
import numpy as np
from numpy import sign, trapz
from teager_py import Teager
import statsmodels.api as sm
import wave

import warnings
warnings.filterwarnings('ignore')

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


In [7]:
# load path
path = pd.read_csv('/project/graziul/ra/team_ser/susas_path.csv')
path.drop(path.iloc[:, 0:1], inplace = True, axis = 1)
path = path[['Path','Arousal','Valence']]

In [8]:
path.head(2)

,Path,Arousal,Valence
0,/project/graziul/data/corpora/susas/speech/act...,High,Negative
1,/project/graziul/data/corpora/susas/speech/act...,High,Negative


In [9]:
# convert librosa to AudioSegment
def librosa_to_audiosegment(filename):
    y, sr = librosa.load(filename)
    # convert from float to uint16
    y = np.array(y * (1<<15), dtype=np.int16)
    audio_segment = pydub.AudioSegment(
        y.tobytes(), 
        frame_rate=sr,
        sample_width=y.dtype.itemsize, 
        channels=1
    )
    return audio_segment

In [21]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

def get_GeMAPS1(filepath):
    audio = librosa_to_audiosegment(filepath) # convert librosa array to audiosegment
    sr = audio.frame_rate # get sample rate
    audio = np.array(audio.get_array_of_samples()) # convert audiosegment to array
    df = smile.process_signal(audio,sr) # get GeMAPS
    features = pd.DataFrame(df.mean(axis=0)).T
    features = features.copy().reset_index()
    features['file'] = filepath
    return features

def get_GeMAPS(path):
    GeMAPS = []
    for i in range(0,len(path)):
        filepath = path.loc[i,'Path']
        # audio,sr = librosa.load(filepath,sr=22050) # all speech tokens were sampled at a rate of 22050Hz
        audio = librosa_to_audiosegment(filepath) # convert librosa array to audiosegment
        sr = audio.frame_rate # get sample rate
        audio = np.array(audio.get_array_of_samples()) # convert audiosegment to array
        df = smile.process_signal(audio,sr) # get GeMAPS
        df['file'] = filepath
        GeMAPS.append(pd.DataFrame(df.mean(axis=0)).T) # take average
        print('processing:',i)
    
    GeMAPS = pd.concat(GeMAPS).reset_index()
    return GeMAPS

In [18]:
filepath = path.loc[0,'Path']
filepath

'/project/graziul/data/corpora/susas/speech/actual/roller/f1/free_oov_all/all1.sph'

In [19]:
get_GeMAPS1 (filepath)

,index,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,shimmerLocaldB_sma3nz,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz,file
0,0,6.292282,-6.76329,18.529961,-0.086114,-0.002062,0.0,0.0,0.0,0.0,0.0,0.0,619.362793,730.045105,-201.0,1487.981445,-201.0,2418.518311,-201.0,/project/graziul/data/corpora/susas/speech/act...


In [ ]:
GeMAPS_0105 = get_GeMAPS(path)

processing: 0
processing: 1
processing: 2
processing: 3
processing: 4
processing: 5
processing: 6
processing: 7
processing: 8
processing: 9
processing: 10
processing: 11
processing: 12
processing: 13
processing: 14
processing: 15
processing: 16
processing: 17
processing: 18
processing: 19
processing: 20
processing: 21
processing: 22
processing: 23
processing: 24
processing: 25
processing: 26
processing: 27
processing: 28
processing: 29
processing: 30
processing: 31
processing: 32
processing: 33
processing: 34
processing: 35
processing: 36
processing: 37
processing: 38
processing: 39
processing: 40
processing: 41
processing: 42
processing: 43
processing: 44
processing: 45
processing: 46
processing: 47
processing: 48
processing: 49
processing: 50
processing: 51
processing: 52
processing: 53
processing: 54
processing: 55
processing: 56
processing: 57
processing: 58
processing: 59
processing: 60
processing: 61
processing: 62
processing: 63
processing: 64
processing: 65
processing: 66
proce

processing: 520
processing: 521
processing: 522
processing: 523
processing: 524
processing: 525
processing: 526
processing: 527
processing: 528
processing: 529
processing: 530
processing: 531
processing: 532
processing: 533
processing: 534
processing: 535
processing: 536
processing: 537
processing: 538
processing: 539
processing: 540
processing: 541
processing: 542
processing: 543
processing: 544
processing: 545
processing: 546
processing: 547
processing: 548
processing: 549
processing: 550
processing: 551
processing: 552
processing: 553
processing: 554
processing: 555
processing: 556
processing: 557
processing: 558
processing: 559
processing: 560
processing: 561
processing: 562
processing: 563
processing: 564
processing: 565
processing: 566
processing: 567
processing: 568
processing: 569
processing: 570
processing: 571
processing: 572
processing: 573
processing: 574
processing: 575
processing: 576
processing: 577
processing: 578
processing: 579
processing: 580
processing: 581
processi

In [ ]:
GeMAPS_0105.head(2)

In [20]:
GeMAPS_0105 = get_GeMAPS(path)
GeMAPS_0105_features = pd.concat([path, GeMAPS_0105.iloc[:,1:]], axis=1)
GeMAPS_0105_features.head(2)

NameError: name 'GeMAPS_0105' is not defined

In [16]:
#GeMAPS_0105 = get_GeMAPS(path)

In [14]:
#GeMAPS_0105_features.to_csv('/project/graziul/ra/team_ser/GeMAPS_0105.csv')

In [ ]:
# feature extraction

In [22]:
# TEO
from teo_cb_auto_env import TeagorExtractor

t = TeagorExtractor()

def get_teo(sample):
    TEO = []
    for i in range(0,len(sample)):
        filepath = sample.iloc[i,0]
        teo_cb_auto_env = t.process_mp3(filepath,1)
        TEO.append(pd.DataFrame(pd.DataFrame(teo_cb_auto_env).mean(axis=0)).T)
    #print('processing:',i)
    TEO = pd.concat(TEO).reset_index()
    return TEO

In [ ]:
TE0_0105 = get_teo(sample)

In [ ]:
features = pd.concat([sample, GeMAPS_0105], axis=1)

In [5]:
filepath = '/home/xuranzeng/SER/ProjectiveNum1.wav'
song = pydub.AudioSegment.from_wav(filepath)
type(song)

pydub.audio_segment.AudioSegment

In [20]:
def split_wav(self, filepath, wav_filepath):
        song = pydub.AudioSegment.from_wav(filepath)
        duration = song.duration_seconds * 1000
        # print(f"Wav file length: {duration}")
        length = 0
        self.audio_data_all = []
        while length + int(self.delta * 1000) <= duration:
            song_piece = song[length:length + int(self.delta * 1000)]
            self.audio_data_all.append(np.array(song_piece.get_array_of_samples()).astype(np.int64))
            length += int(self.delta * 1000)
        # print(len(self.audio_data_all))